In [27]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import warnings

In [28]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow

In [29]:
tensorflow.test.is_gpu_available()

False

In [30]:
#set seed value
seed = 1842
tensorflow.random.set_seed(seed)
np.random.seed(seed)
#turn off warning
warnings.simplefilter('ignore')

### Loading data

In [31]:
#define the image dataset

image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)

#train
train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                   directory='data_cleaned/Train',
                                                   shuffle=True,
                                                   target_size=(224,224),
                                                   subset="training",
                                                   class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=32,
                                                   directory='data_cleaned/Train',
                                                   shuffle=True,
                                                   target_size=(224,224),
                                                   subset="validation",
                                                   class_mode='categorical')

#organize data from our prediction
image_generator_submission = ImageDataGenerator(rescale=1/255)
submission = image_generator_submission.flow_from_directory(directory='data_cleaned/scraped_images',
                                                            shuffle=False,
                                                            target_size=(224,224),
                                                            class_mode=None)

Found 430 images belonging to 2 classes.
Found 106 images belonging to 2 classes.
Found 100 images belonging to 1 classes.


## Build first classifier(CNN)

In [32]:
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu', input_shape=(224,224,3)),
    #keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(2, activation='softmax')
    
])

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                        patience=3,
                                        restore_best_weights=True)


In [33]:
model.fit(train_dataset, epochs=20, validation_data=validation_dataset, callbacks=callback)

Epoch 1/20
14/14 [==============================] - 13s 843ms/step - loss: 8.0052 - accuracy: 0.5977 - val_loss: 0.2244 - val_accuracy: 0.9151
Epoch 2/20
14/14 [==============================] - 12s 820ms/step - loss: 1.3056 - accuracy: 0.8163 - val_loss: 0.1451 - val_accuracy: 0.9528
Epoch 3/20
14/14 [==============================] - 12s 822ms/step - loss: 0.4918 - accuracy: 0.8791 - val_loss: 0.1615 - val_accuracy: 0.9528
Epoch 4/20
14/14 [==============================] - 11s 808ms/step - loss: 0.1666 - accuracy: 0.9372 - val_loss: 0.0759 - val_accuracy: 0.9717
Epoch 5/20
14/14 [==============================] - 12s 818ms/step - loss: 0.0974 - accuracy: 0.9628 - val_loss: 0.0818 - val_accuracy: 0.9717
Epoch 6/20
14/14 [==============================] - 12s 821ms/step - loss: 0.0187 - accuracy: 0.9977 - val_loss: 0.1861 - val_accuracy: 0.9340
Epoch 7/20
14/14 [==============================] - 11s 814ms/step - loss: 0.0085 - accuracy: 1.0000 - val_loss: 0.0821 - val_accuracy: 0.9717

In [34]:
loss,accuracy = model.evaluate(validation_dataset)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

4/4 [==============================] - 1s 291ms/step - loss: 0.0759 - accuracy: 0.9717
Loss:  0.07591324299573898
Accuracy:  0.9716981053352356
